# reader

> Necessary scripts to read orbits from different formats

In [ ]:
#| default_exp reader

In [ ]:
#| export
import h5py
from scipy.io import loadmat
import numpy as np
from pathlib import Path
import pandas as pd

In [ ]:
from unittest.mock import patch, MagicMock

## Loading Data

This function loads orbit data from .mat or .h5

In [ ]:
#| export
def load_orbit_data(file_path, variable_name=None, dataset_path=None):
    """
    Load orbit data from MATLAB .mat files or HDF5 .h5 files.

    Parameters:
    - file_path: str, the path to the .mat or .h5 file.
    - variable_name: str, optional, the name of the variable in the .mat file.
    - dataset_path: str, optional, the path to the dataset in the .h5 file.

    Returns:
    - data: The loaded orbit data.
    """
    if file_path.endswith('.mat'):
        if variable_name is None:
            raise ValueError("variable_name must be provided for .mat files")
        mat = loadmat(file_path)
        if variable_name in mat:
            data = mat[variable_name]
        else:
            raise ValueError(f"{variable_name} not found in {file_path}")
    
    elif file_path.endswith('.h5'):
        with h5py.File(file_path, 'r') as file:
            if dataset_path:
                if dataset_path in file:
                    data = np.array(file[dataset_path])
                else:
                    raise ValueError(f"{dataset_path} not found in {file_path}")
    
    else:
        raise ValueError("Unsupported file format. Please provide a .mat or .h5 file.")
    
    return data

In [ ]:
#| test "load_orbit_data with .mat file"
mock_mat_data = {'Xarray': np.array([1, 2, 3])}
mock_h5_data = np.array([4, 5, 6])

# Test for load_orbit_data with .mat file
with patch('__main__.loadmat', return_value=mock_mat_data) as mock_loadmat:
    result = load_orbit_data('test_data.mat', variable_name='Xarray')
    assert (result == mock_mat_data['Xarray']).all(), "MAT file loading failed or data mismatch"
    mock_loadmat.assert_called_once_with('test_data.mat')

# Test for load_orbit_data with .h5 file
with patch('__main__.h5py.File') as mock_h5py:
    mock_file = MagicMock()
    mock_file.__enter__.return_value = {'/files/PERIODIC ORBITS': mock_h5_data}
    mock_h5py.return_value = mock_file
    result = load_orbit_data('test_data.h5', dataset_path='/files/PERIODIC ORBITS')
    assert (result == mock_h5_data).all(), "H5 file loading failed or data mismatch"

### Functions to return data directly

In [ ]:
#| export
def get_orbit_features(data_directory=None):
    """
    Load orbit feature data from a MAT file located in the specified directory or a default 'Data' directory.

    :param data_directory: Optional; The directory where the MAT file is stored. If not specified, 
                           the function assumes a default directory relative to the script's location.
    :type data_directory: pathlib.Path, optional
    
    :return: A DataFrame containing detailed orbit features such as family, positions, velocities, 
             Jacobi constant, period, and stability index.
    :rtype: pandas.DataFrame
    """

    # Set default data directory if not provided
    if data_directory is None:
        data_directory = Path(__file__).parent / "Data"
    
    # Path to the MAT file containing orbit features
    feat_route = data_directory / "EM_IC_ARRAY.mat"
    
    # Load data from the MAT file
    feat = load_orbit_data(feat_route, variable_name='out_EM')
    
    # Define column labels for the DataFrame
    column_labels = [
        'Orbit Family', 'Initial Position X', 'Initial Position Y', 'Initial Position Z',
        'Initial Velocity X', 'Initial Velocity Y', 'Initial Velocity Z',
        'Jacobi Constant', 'Period', 'Stability Index'
    ]
    
    # Create a DataFrame from the loaded data
    features = pd.DataFrame(feat, columns=column_labels)

    return features


In [ ]:
#| export
def get_orbit_data_and_labels(data_directory=None):
    """
    Load orbit data and corresponding labels from predefined locations relative to the script's execution directory.

    The function loads a specific dataset from an HDF5 file and a set of labels from a MAT file, both expected to be 
    in a predefined directory structure. This setup is intended for demonstration or testing purposes, where the file 
    structure and the dataset paths are known ahead of time.

    Returns:
        tuple: 
            - orbit_data (numpy.ndarray): Array containing the reshaped orbit data from the HDF5 file. 
              The data is reshaped and transposed to match the desired format for analysis.
            - labels (pandas.Series): Series containing the orbit classification labels repeated to match 
              the reshaped data.

    The reshaped data array involves transposing dimensions to reorganize the data for further processing.
    Each row in the Series corresponds to a specific orbit with its classification.
    """

    # Get Orbit Data
    if data_directory is None:
        data_directory = Path(__file__).parent / "Data"
    hdf5_file_path = data_directory / "em_orbits.h5"
    dataset_path = '/files/PERIODIC ORBITS'
    
    data = load_orbit_data(str(hdf5_file_path), dataset_path=dataset_path)

    # Reshape and transpose data for further processing
    reshaped_array = data.reshape(36071, 7, 5, 1500)
    orbit_data = reshaped_array.transpose(0, 2, 1, 3).reshape(36071 * 5, 7, 1500)
    
    # Get Orbit Classes using the refactored function
    features = get_orbit_features(data_directory)
    # Extract and replicate the 'Orbit Family' labels
    labels = features['Orbit Family'].repeat(5).reset_index(drop=True)

    return orbit_data, labels

In [ ]:
#| export
def get_example_orbit_data():
    """
    Load orbit data from a hardcoded MAT file located in the `data` directory.
    
    The function is specifically designed to load the 'Xarray' variable 
    from the '1_L2_S_200_EM_CR3BP.mat' file. This setup is intended for 
    demonstration or testing purposes, where the data file and the variable 
    of interest are known ahead of time.

    :return: A numpy.ndarray containing the transposed data from the MAT file.
    """
    # Hardcoded file name and variable name
    filename = "1_L2_S_200_EM_CR3BP.mat"
    variable_name = 'Xarray'
    
    # Assuming the notebook or script is executed in a directory at the same level as the `data` folder
    current_dir = Path.cwd().parent
    matlab_file_path = current_dir / "data" / filename
    
    # Assuming `load_orbit_data` is a predefined function that loads and returns data from the .mat file
    data_mat = load_orbit_data(str(matlab_file_path), variable_name=variable_name)
    # Transpose the data for further use
    data_mat = np.transpose(data_mat, (2, 1, 0))
    
    return data_mat


In [ ]:
data = get_example_orbit_data()
data.shape

(200, 6, 300)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()